# PyARPES の使い方（私家版）

## はじめに。

[pyarpes](https://arpes.readthedocs.io/)といpython ライブラリがある。
かなり不完全なところがあって、バギーな関数もある様だけど、一応なりとも使える（かな）。
もっともIgorで行う解析と何が違うのか？という疑問はあって、結局Igorでやることになるとは思うのだけど、大量データを扱うことを考えたときに便利かもしれない。
というか、便利なのが分かったらIgor procedureを書かないといけないので、そのプロトタイプと考えると良いのかもしれない。
基本的に xarray を拡張したファイルを取り扱う設計で、 xarray そのものは安定しているようなので、信頼性は比較的高い。


## autoreload 関係。
最終的にはここは無くてもいい。

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## import 

以下の様にしておこう。

* まあ numpy はいるだろし。
* matplotlib もあらわに出しておいた方が、チューニングしやすい。
* ARPES データをLogスケールで色付けする可能性があるので。LogNormを導入しておく。
* arpes.config は（おそらく）必須。
* pes.SPD_main import SPDEndstation は僕が書いた。（ので読むディレクトリが apres. 以下ではない）
* qt_tool は PyQtを利用した、データ表示。
* ktool は　...
* convert_to_kspace は　角度-> 運動量変換（ただ、どこまでデータの内容を考えて変換しようとしているのか、現在（2022/05/09）まだ未決。おそらくは、Binding energyとして処理をすすめることを前提しているようだ。
* hvplot.xarray は無くても良いかもしれない。Bokeh を使ってみたいときに。

以下のライブラリをロードしてエラーがでないぐらいには、ライブラリをインストールしておく必要があることに注意。

PyQt5を要求している。

* (pip ではインストール失敗するので、Brew　でインストールする必要がある）。
* ```brew install pyqt5``` の後

```bash
ln -s /opt/homebrew/Cellar/pyqt@5/${pyqt_version}/lib/python${python_version}/site-packages/*  $(dirname $(which python))/../lib/python${python_version}/site-packages/
```
といった処理が必要かもしれない。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import arpes.config
from arpes.io  import load_data
from pes.SPD_main import SPDEndstation
from arpes.plotting.qt_tool import qt_tool
from arpes.plotting.qt_ktool import ktool
from arpes.utilities.conversion import convert_to_kspace
import hvplot.xarray

## データのロード

自分で測定したデータを読み込むために、
* from arpes.io  import load_data 

が必須。

自分のデータの内容に不安を覚えたら、
* from arpes.io import example_data

を使い、

```python:
ex = example_data.cut
```

として、　ex の中身を見て比べてみると何かヒントがあるかもしれない。


In [ ]:
from arpes.io import example_data

load_data は以上のような関数で、実際に使うだけなら、そんなにむずかしくない。単純に

In [ ]:
f=load_data("/Users/arafune/src/analysis_tool/test/pes/data/PES_16_Spectrum_3.itx",location="SPD_phoibos")
# f2 = load_data("/Users/arafune/src/analysis_tool/test/pes/GrIr.sp2", location="SPD_phoibos")

でよい。ただし、

* x, y, z 軸の値や 角度βを記録に含めたい。

など、itx ファイルや、 sp2ファイルに記載されない情報も 付加させたいときは。

```python
f=load_data("/Users/arafune/src/analysis_tool/test/pes/PES_16_Spectrum_3.itx",location="SPD_phoibos",
             x=1,
             sample_name="Cu(111)")
```

などと、することができる。

''x'' や ''sample_name''　などの変数名は予め定義されているのでそれを使うように推奨される。（そうでないと、ただのコメントにしかならない）
どのような変数名が公式なものかは、どこにもリストらしきものはないので、実際に読み込んだ後で、チェックする必要があるかもしれない。

* sample_name
* sample_workfunction 

がちょっと知らないと使えない変数名。 ("sample" ではないし、 "workfunction"ではない。（アナライザーの仕事関数として予約されている） )
このマニュアルでは（少なくともこのセクションでは）付加情報は加えていない。

``xarray_extensions.py`` を読むしかない。

In [ ]:
f.S.polarization

In [ ]:
f.S.history

In [ ]:
f.S.spectrometer_settings

In [ ]:
f.S.experimental_conditions

In [ ]:
f.S.scan_info

In [ ]:
f.S.sample_angles

In [ ]:
f.S.sample_pos

In [ ]:
f.S.experiment_info

In [ ]:
f.S.laser_info

In [ ]:
f.S.probe_info

In [ ]:
f.S.pump_info

In [ ]:
f.S.analyzer_info

## Plot

機能としてはシンプル。この辺は、 pyarpes の機能ではなく、 xarray の機能と考えるのが無難。
上述のごとくxarrayは比較的良く整備されているように見える。
ま、ただのデータ構造なので、メンテがそれほど難しくないのかもしれない。

### matplotlib base

In [ ]:
f.S.sample_work_function

In [ ]:
f.S.plot()

以下ようにすると、matplotlib.pyplot を直接使う事ができるように見える。

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
f.S.spectrum.T.plot(ax=ax)

しかし、xarray の plot method は matplotlib の thin wrapperなだけなので、出力フォーマットを変えるのであれば、以下のようにする方がよりよい。

In [ ]:
f.S.spectrum.T.plot(aspect=.7, size=6, cmap=plt.cm.plasma)
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

In [ ]:
# Logarithmic scale
f.S.spectrum.T.plot(aspect=.7, size=4, cmap=plt.cm.jet, norm=LogNorm(vmin=1, vmax=100))
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("Emission Angle ( rad )")
plt.title("Xe/Au(111) monochromatic 2PPE")
plt.show()

### Bokeh base 

Bokeh を使って、インタラクティブな可視化をすることもできる。直接Bokeh を使っても良いはずだろうとは思うけど、ここではhvplot を使ってみる。
~~ただし、hvplot の出力はvscode で見ることができない様なので注意。~~ 

In [ ]:
f.hvplot.image(colorbar=True)

In [ ]:
f.S.spectrum.T.hvplot(invert=True, frame_height=400, frame_width=200)

## k-space 変換

`convert_to_kspace` というそのものズバリの関数がある。以下の例では、 `kp` を指定してあるが、もし指定しないと適切な範囲を自動的に決めてくれる。ただ、エネルギー軸をどうするかはこれから検証する必要がある。

In [ ]:
convert_to_kspace(f.spectrum, kp=np.linspace(-1,1,100)).plot()
plt.ylabel("Energy relative to E_F ( eV )")
plt.xlabel("$k_{//}$")

## まだよく分かってないこと。

* Fermi level correction から アナライザーの misalignment に由来するズレを解消できるか？
* attrs.offset や beta angle の設定は自動的に考慮されてしまうのか？

いずれにしても、これは個々のケースバイケースの話なので、今回はふれない（ MoTe2 ARPES をこのパッケージで解析すると自動的に上の二つの問題には直面することになる・・・）